# Azure AI Agent with Existing Thread Example

This notebook demonstrates working with pre-existing conversation threads by providing thread IDs for thread reuse patterns.

## Features Covered:
- Creating and managing persistent conversation threads
- Working with existing thread IDs
- Thread lifecycle management (create, use, delete)
- Conversation continuity across sessions
- Thread initialization and validation

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

This example demonstrates how to work with existing threads to maintain conversation continuity.

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [ ]:
import os
from pathlib import Path
from azure.identity import AzureCliCredential, InteractiveBrowserCredential
from dotenv import load_dotenv  # For loading environment variables from .env file
import asyncio
from random import randint
from typing import Annotated
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from pydantic import Field

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../../.env')  # Load environment variables from .env file

# Get connection string and model deployment for authentication
conn_string = os.environ.get("PROJECT_CONNECTION_STRING")
model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

print(f"🔗 Using Connection String: {conn_string[:50]}..." if conn_string else "❌ No connection string found")
print(f"🤖 Using Model: {model_deployment}")
print("✅ Environment variables loaded successfully")

## Check Environment Variables

Let's verify that the required environment variables are set:

In [ ]:
# Check required environment variables
conn_string_check = os.getenv('PROJECT_CONNECTION_STRING')
model_deployment_check = os.getenv('MODEL_DEPLOYMENT_NAME')

if conn_string_check:
    print(f"✅ PROJECT_CONNECTION_STRING: {conn_string_check[:50]}...")
else:
    print("❌ PROJECT_CONNECTION_STRING: Not set")
    print("⚠️  Please set the PROJECT_CONNECTION_STRING environment variable")

if model_deployment_check:
    print(f"✅ MODEL_DEPLOYMENT_NAME: {model_deployment_check}")
else:
    print("❌ MODEL_DEPLOYMENT_NAME: Not set, using default 'gpt-4o'")

## Define Function Tools

Let's define a simple weather function that our agent can use:

In [ ]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

## Create and Use Existing Thread

This example shows how to:
1. Create a thread that persists beyond the current session
2. Use the thread ID to continue conversations
3. Properly clean up thread resources

In [ ]:
async def main() -> None:
    print("=== Azure AI Chat Client with Existing Thread ===")

    # Use AzureCliCredential for authentication
    credential = AzureCliCredential()
    
    # Create the agent client using PROJECT_CONNECTION_STRING
    async with AzureAIAgentClient(
        project_endpoint=conn_string,
        async_credential=credential,
        model_deployment_name=model_deployment
    ) as chat_client:
        
        # Create an agent
        agent = chat_client.create_agent(
            name="WeatherThreadAgent",
            instructions="You are a helpful weather agent that can maintain conversation context.",
            tools=get_weather,
        )
        
        print(f"✅ Created agent: WeatherThreadAgent")

        try:
            # Create a new thread for the conversation
            thread = agent.get_new_thread()
            print(f"✅ Created new thread: {thread.service_thread_id}")
            
            # First query
            result1 = await agent.run("What's the weather like in Tokyo?", thread=thread)
            print(f"Result 1: {result1}\n")
            
            # Second query in the same thread (maintains context)
            result2 = await agent.run("How about compared to yesterday?", thread=thread)
            print(f"Result 2: {result2}\n")
            
            # Third query in the same thread
            result3 = await agent.run("What should I wear for this weather?", thread=thread)
            print(f"Result 3: {result3}\n")
            
        except Exception as e:
            print(f"❌ Error running agent: {str(e)}")
            
        print(f"💡 Thread conversation completed successfully")

## Execute the Example

Run the main function to see the existing thread workflow:

In [ ]:
# Run the main function
await main()

## Multi-Turn Conversation Example

This example shows how to have a multi-turn conversation using a persistent thread:

In [ ]:
async def multi_turn_conversation_example():
    """Example showing multi-turn conversation with persistent thread."""
    print("=== Multi-Turn Conversation Example ===")
    
    # Use AzureCliCredential for authentication
    credential = AzureCliCredential()
    
    async with AzureAIAgentClient(
        project_endpoint=conn_string,
        async_credential=credential,
        model_deployment_name=model_deployment
    ) as chat_client:
        
        # Create an agent for multi-turn conversation
        agent = chat_client.create_agent(
            name="ConversationAgent",
            instructions="You are a helpful weather agent. Remember previous conversations and provide contextual responses.",
            tools=get_weather,
        )
        
        print(f"✅ Created agent: ConversationAgent")
        
        try:
            # Create a persistent thread for the conversation
            thread = agent.get_new_thread()
            print(f"📝 Started conversation with thread: {thread.service_thread_id}")
            
            # Multi-turn conversation
            conversation = [
                "What's the weather like in Paris?",
                "How about in London?", 
                "Which city has better weather between the two I just asked about?",
                "Thank you for the information about Paris and London."
            ]
            
            for i, user_message in enumerate(conversation, 1):
                print(f"\n--- Turn {i} ---")
                print(f"🤔 User: {user_message}")
                
                result = await agent.run(user_message, thread=thread)
                print(f"🤖 Agent: {result}")
                
        except Exception as e:
            print(f"❌ Error in multi-turn conversation: {str(e)}")
            
        print(f"\n💡 Multi-turn conversation completed successfully")

# Run the multi-turn conversation example
await multi_turn_conversation_example()

## Thread Management Example

This example demonstrates advanced thread management operations:

In [ ]:
async def thread_management_example():
    """Advanced example showing thread management operations."""
    print("=== Thread Management Example ===")
    
    # Use AzureCliCredential for authentication
    credential = AzureCliCredential()
    
    async with AzureAIAgentClient(
        project_endpoint=conn_string,
        async_credential=credential,
        model_deployment_name=model_deployment
    ) as chat_client:
        
        # Create an agent for thread management
        agent = chat_client.create_agent(
            name="ThreadManagerAgent",
            instructions="You are a helpful assistant. Each thread represents a different conversation topic.",
            tools=get_weather,
        )
        
        print(f"✅ Created agent: ThreadManagerAgent")
        
        try:
            # Create multiple threads for different topics
            threads = []
            for i in range(3):
                thread = agent.get_new_thread()
                threads.append(thread)
                print(f"📝 Created thread {i+1}: {thread.service_thread_id}")
            
            # Use each thread for a different topic
            topics = [
                ("Weather in New York", "What's the weather in New York?"),
                ("Weather in Berlin", "How's the weather in Berlin?"),
                ("Weather comparison", "Compare weather in different regions")
            ]
            
            for i, (topic, query) in enumerate(topics):
                print(f"\n--- Thread {i+1}: {topic} ---")
                
                print(f"🤔 User: {query}")
                result = await agent.run(query, thread=threads[i])
                print(f"🤖 Agent: {result}")
                
            # Follow up on first thread to show conversation continuity
            print("\n--- Following up on Thread 1 ---")
            followup = "Is it good weather for outdoor activities?"
            print(f"🤔 User: {followup}")
            result = await agent.run(followup, thread=threads[0])
            print(f"🤖 Agent: {result}")
            
        except Exception as e:
            print(f"❌ Error in thread management: {str(e)}")
            
        print(f"\n💡 Thread management example completed successfully")

# Run the thread management example
await thread_management_example()

## Key Takeaways

1. **Persistent Threads**: Threads can persist beyond a single session, enabling conversation continuity
2. **Thread Initialization**: Always verify that threads are properly initialized before use
3. **Conversation Context**: Threads maintain conversation history, allowing for contextual responses
4. **Resource Management**: Always clean up threads when they're no longer needed
5. **Multi-Threading**: You can manage multiple separate conversations using different thread IDs
6. **Thread Reuse**: Threads can be reused across different agent instances

## Best Practices

1. **Thread Lifecycle**: Create threads when starting conversations and delete them when finished
2. **Error Handling**: Use try-finally blocks to ensure thread cleanup
3. **Thread Validation**: Always check if threads are initialized before using them
4. **Context Awareness**: Leverage thread history for more contextual conversations
5. **Resource Monitoring**: Monitor thread usage to avoid unnecessary resource consumption

## Use Cases

- **Customer Support**: Maintaining conversation history across support sessions
- **Multi-Session Chats**: Continuing conversations across different user sessions
- **Conversation Branching**: Managing multiple conversation topics simultaneously
- **Context Preservation**: Keeping conversation context for better user experience
- **Session Management**: Managing different conversation sessions for different users